In [1]:
# 將一部影片轉換成demo影片
import argparse
import time
import os
import numpy as np
import pandas as pd
import functools
import operator
import requests
import base64

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow import keras

from tf_pose import common
import cv2
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

import queue
import threading

class MyArgs():
    def __init__(self):
        self.video = './mdata/videos/BedTest.mp4'
        self.model = 'mobilenet_thin'
        self.resize = '0x0'
        self.resize_out_ratio = 4.0
        self.fall_model = 'mdata/model/model_2'
        self.notify_url_origin = 'https://fall-detection-line-notify.406.csie.nuu.edu.tw'
args = MyArgs()

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# ==== [ load pre-train model ] ======================
sess = tf.Session()
graph = tf.get_default_graph()
keras.backend.set_session(sess)
model = keras.models.load_model(args.fall_model)
model.predict(np.ones((1, 54)))
# model._make_predict_function()
print('================== [ Model loaded ]')

# ==== [ load tf-pose model ] ========================
print('================== [ Init of Tf-Pose-Estimator Start ]')
w, h = model_wh(args.resize)
if w == 0 or h == 0:
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(432, 368))
else:
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))
print('================== [ Init of Tf-Pose-Estimator Finished ]')

[2020-11-11 15:56:12,033] [TfPoseEstimator] [INFO] loading graph from /home/jovyan/work/fall-detection/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)
2020-11-11 15:56:12,033 INFO loading graph from /home/jovyan/work/fall-detection/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)


================== [ Model loaded ]
================== [ Init of Tf-Pose-Estimator Start ]
dense_1_input
dense_1/kernel/Initializer/random_uniform/shape
dense_1/kernel/Initializer/random_uniform/min
dense_1/kernel/Initializer/random_uniform/max
dense_1/kernel/Initializer/random_uniform/RandomUniform
dense_1/kernel/Initializer/random_uniform/sub
dense_1/kernel/Initializer/random_uniform/mul
dense_1/kernel/Initializer/random_uniform
dense_1/kernel
dense_1/kernel/IsInitialized/VarIsInitializedOp
dense_1/kernel/Assign
dense_1/kernel/Read/ReadVariableOp
dense_1/bias/Initializer/zeros
dense_1/bias
dense_1/bias/IsInitialized/VarIsInitializedOp
dense_1/bias/Assign
dense_1/bias/Read/ReadVariableOp
dense_1/MatMul/ReadVariableOp
dense_1/MatMul
dense_1/BiasAdd/ReadVariableOp
dense_1/BiasAdd
dense_1/Tanh
dense_2/kernel/Initializer/random_uniform/shape
dense_2/kernel/Initializer/random_uniform/min
dense_2/kernel/Initializer/random_uniform/max
dense_2/kernel/Initializer/random_uniform/RandomUniform
d

In [3]:
# ==== [ main ] ======================================

def imageToHuman(img, draw = False):
    global e
    try:
        startTime = time.time()
        humans = e.inference(img, resize_to_default=(w > 0 and h > 0), upsample_size=args.resize_out_ratio)
        # print('======== [ Handle image in %.4f seconds ]' % (time.time() - startTime))

        img2 = None
        if draw:
            img2 = TfPoseEstimator.draw_humans(img, humans, imgcopy=False)

        return (humans, img2)
    except Exception as e:
        cv2.waitKey()
        return []

HUMAN_POINTS_NUMS = 18
TARGET_FALL = 0
IMAGE_DIM = (720, 405)

def listFlat(l):
    return functools.reduce(operator.iconcat, l, [])

def handleHumanData(h):
    tmp_data = []
    for i in range(0, HUMAN_POINTS_NUMS):
        if i in h.body_parts.keys():
            p = h.body_parts[i]
            x = p.x
            y = p.y
            score = p.score
        else:
            x = np.nan
            y = np.nan
            score = np.nan
        tmp_data.append({ 'x': x, 'y': y, 'score': score })
    df = pd.DataFrame(tmp_data)
    df['x'] = df['x'] - df['x'].mean()
    df['y'] = df['y'] - df['y'].mean()
    df['x'].fillna(value=0, inplace=True)
    df['y'].fillna(value=0, inplace=True)
    df['score'].fillna(value=df['score'].mean(), inplace=True)
    return listFlat(df.values)

def imageFallDetection(img, window_size):
    # jpeg_flag = [int(cv2.IMWRITE_JPEG_QUALITY), 50]
    img = cv2.resize(img, IMAGE_DIM, interpolation=cv2.INTER_AREA)
    # result, encimg = cv2.imencode('.jpg', image, jpeg_flag)
    # decimg = cv2.imdecode(encimg, 1)
    img = cv2.GaussianBlur(img, window_size, 0)
    # (T, img) = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)

    (humans, img2) = imageToHuman(img, draw=True)
    for i, h in enumerate(humans):
        if len(h.body_parts.keys()) > HUMAN_POINTS_NUMS/2:
            data = handleHumanData(h)
            data = np.array(data)
            data = data.reshape((1, 54))

            with graph.as_default():
                keras.backend.set_session(sess)
                res = model.predict_classes(data)
                if res[0] == TARGET_FALL:
                    return (True, img2)
    return (False, img2)

NOTIFY_URL = args.notify_url_origin + '/api/notify';
def fallDetectionNotify(img, msg = None):
    # token = '7QI2XKUNR95IAB1SiKSLCDNaqJIrZoz0Kx1zH8HWH2T'
    # headers = {
    #     "Authorization": "Bearer " + token,
    #     "Content-Type" : "application/x-www-form-urlencoded"
    # }
    # payload = {'message': msg}
    # r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    _, imdata = cv2.imencode('.jpg', img)
    imdata = base64.b64encode(imdata).decode()
    data = { 'image': imdata }
    if msg is not None:
        data['message'] = msg
    r = requests.post(NOTIFY_URL, json=data)
    return r.status_code

CONFIG_URL = args.notify_url_origin + '/api/config';
def getWebConfig():
    print('================== [ GET web-config from url: ' + CONFIG_URL + '... ]')
    r = requests.get(CONFIG_URL)
    data = r.json()
    if data['success']:
        print('================== [ GET web-config successfully. ]')
        return data['data']
    print('================== [ GET web-config false. ]')
    return None


def FallDetection(video_path):
    if '.mp4' not in video_path:
        print('The video of arguments must be MP4.')
        return
#     print('========== [ start Handle ]')
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    
    video_size = (frame.shape[1], frame.shape[0])
    print('================== [ Original Image: (%d, %d) ]' % video_size)
    print('================== [ FPS of video: %f ]' % video_fps)

    print('================== [ Test notify api... ]')
    print('================== [ notify api response status code: %d ]' % fallDetectionNotify(frame, 'Start up'))

    GaussianBlur_WINDOW_SIZE = (17, 17)
    UPDATE_WEB_CONFIG_INTERVAL = 30
    WEB_CONFIG = getWebConfig()
    if WEB_CONFIG is not None:
        GaussianBlur_WINDOW_SIZE = (WEB_CONFIG['gaussian_blur_window_size'], WEB_CONFIG['gaussian_blur_window_size'])
    last_update_web_config_time = time.time()

    FPS_FIX = 5            # 原始FPS為15，一秒取(FPS/FIX_FIX)張
    DETECTION_THRESHOLD = 3 # 連續偵測到幾次才判斷跌倒
    INTERVAL_MINIMUM = 0    # 至少隔幾秒才能再發送一次通知。由於發送通知的server端已經有設定限制時間，故這邊不再設定。

    # SAVE_PATH = './mdata/detection_images/t' + str(time.time()).split('.')[0]
    # os.mkdir(SAVE_PATH)
    
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    output_video = cv2.VideoWriter(video_path.replace('.mp4', '_output.mp4'), fourcc, (video_fps / FPS_FIX), IMAGE_DIM)
    
    count = -1
    detection_count = 0
    fall_img_count = 0
    
    start_handle_time = time.time()
    print('========== [ Start handle video ]')
    last_time = time.time()
    frame_count = 0
    while ret:
        (ret, img) = cap.read()
        
        cur_time = time.time()

#         if cur_time - last_update_web_config_time > UPDATE_WEB_CONFIG_INTERVAL:
#             WEB_CONFIG = getWebConfig()
#             if WEB_CONFIG is not None:
#                 GaussianBlur_WINDOW_SIZE = (WEB_CONFIG['gaussian_blur_window_size'], WEB_CONFIG['gaussian_blur_window_size'])
#             last_update_web_config_time = time.time()

        count = count + 1
        if count % FPS_FIX != 0:
            continue

        if img is not None:
            (is_fall, img2) = imageFallDetection(img, GaussianBlur_WINDOW_SIZE)

            if is_fall:
                detection_count = detection_count + 1
                # print('======== [ Fall......%d ]' % detection_count)
                if detection_count >= DETECTION_THRESHOLD:
                    cv2.rectangle(img2, (620, 10), (710, 60), (255, 255, 255), -1)
                    cv2.putText(img2, 'Fall', (635, 45), 4, 1, (0, 0, 255), 1, cv2.LINE_AA)

                    if cur_time - last_time > INTERVAL_MINIMUM:
                        # print('======== [ Fall Detection ]')
                        img = cv2.resize(img, (720, 405), interpolation=cv2.INTER_AREA)
                        #fallDetectionNotify(img)
                        #cv2.imwrite((SAVE_PATH + "/img_%d.jpg") % fall_img_count, img)
                        fall_img_count = fall_img_count + 1
                    detection_count = 0
                    last_time = cur_time
                    # return
                else:
                    cv2.rectangle(img2, (620, 10), (710, 60), (255, 255, 255), -1)
                    cv2.putText(img2, 'Fall', (635, 45), 4, 1, (255, 0, 0), 1, cv2.LINE_AA)
            else: # 要連續偵測到才算，而非累加
                detection_count = 0

            # cv2.imshow('Tf Pose Preview', img2)
            frame_count = frame_count + 1
            print('Handled Frames: %d' % frame_count, end="\r")
            output_video.write(img2)
        else:
            print('\n======== [ Frame is empty... ]')

        if cv2.waitKey(1) == 27:
            break
    # cv2.destroyAllWindows()
    print('')
    cap.release()
    output_video.release()
    print('======== [ Handle video in %.4f seconds ]' % (time.time() - start_handle_time))

def start(video_path):
    FallDetection(video_path)

start(args.video)
# start('rtsp://120.105.129.229:1554/stream2')

================== [ Original Image: (2304, 1296) ]
================== [ FPS of video: 14.250000 ]
================== [ Test notify api... ]
================== [ notify api response status code: 200 ]
================== [ GET web-config from url: https://fall-detection-line-notify.406.csie.nuu.edu.tw/api/config... ]
================== [ GET web-config successfully. ]
========== [ Start handle video ]
Handled Frames: 3777
======== [ Handle video in 494.3333 seconds ]
